In [188]:
import re
import pandas as pd
import numpy as np
from time import time
import gensim 
from gensim.models import Word2Vec 

In [191]:
#read data files downloaded from web
data = pd.read_csv("/home/mahmoud/Desktop/Tunisian Dialect Language Model TDLM/corpus/corpus.txt",
                   header=None,sep='\n',error_bad_lines=False)
data.columns = ["text"]
data.head()

b'Skipping line 29: expected 1 fields, saw 2\nSkipping line 259: expected 1 fields, saw 2\nSkipping line 6339: expected 1 fields, saw 2\nSkipping line 6341: expected 1 fields, saw 2\nSkipping line 6342: expected 1 fields, saw 2\nSkipping line 6636: expected 1 fields, saw 2\nSkipping line 7431: expected 1 fields, saw 2\nSkipping line 7536: expected 1 fields, saw 2\nSkipping line 7537: expected 1 fields, saw 2\nSkipping line 7687: expected 1 fields, saw 2\nSkipping line 7917: expected 1 fields, saw 2\nSkipping line 8387: expected 1 fields, saw 2\nSkipping line 8972: expected 1 fields, saw 2\nSkipping line 9963: expected 1 fields, saw 2\nSkipping line 13698: expected 1 fields, saw 2\nSkipping line 13703: expected 1 fields, saw 2\nSkipping line 14709: expected 1 fields, saw 2\nSkipping line 14812: expected 1 fields, saw 2\nSkipping line 14847: expected 1 fields, saw 2\nSkipping line 14879: expected 1 fields, saw 2\nSkipping line 15018: expected 1 fields, saw 2\nSkipping line 15036: expecte

,text
0,يا سيدي علي عيشوشه واله تمثل مليح وتمثيلها مقنع
1,امنه تصلح وزيره مراه عالاقل تعطي منظر باهي لتو...
2,حنان الشقراني احسن
3,بري روحي
4,معلم يا لطفي


In [192]:
def count_tokens(row):
    return len(row.split())

data["text"]  = data["text"].apply(str)
data["text"]  = data["text"].apply(str.strip)

data["Number_of_tokens"]  = data["text"].apply(count_tokens)

In [193]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
Number_of_tokens,52603.0,6.61468,6.359828,1.0,3.0,5.0,8.0,261.0


In [194]:
def tokenize(x):
    return str(x).split()


data["words"] = data["text"].apply(tokenize)

print(data.shape)
data.head()

(52603, 3)


,text,Number_of_tokens,words
0,يا سيدي علي عيشوشه واله تمثل مليح وتمثيلها مقنع,9,"[يا, سيدي, علي, عيشوشه, واله, تمثل, مليح, وتمث..."
1,امنه تصلح وزيره مراه عالاقل تعطي منظر باهي لتو...,14,"[امنه, تصلح, وزيره, مراه, عالاقل, تعطي, منظر, ..."
2,حنان الشقراني احسن,3,"[حنان, الشقراني, احسن]"
3,بري روحي,2,"[بري, روحي]"
4,معلم يا لطفي,3,"[معلم, يا, لطفي]"


In [195]:
def keep_letters_only(text):  
    text = re.sub("à","a",text)
    text = re.sub("é","e",text)
    text = re.sub("è","e",text)
    text = re.sub("ù","u",text)
    text = text.lower()
    return text

data["text"] = data["text"].apply(keep_letters_only)

In [196]:
corpus = []
def to_corpus(row):
    corpus.append(row.split())
    return row

data["text"].apply(to_corpus)
#corpus

0          يا سيدي علي عيشوشه واله تمثل مليح وتمثيلها مقنع
1        امنه تصلح وزيره مراه عالاقل تعطي منظر باهي لتو...
2                                       حنان الشقراني احسن
3                                                 بري روحي
4                                             معلم يا لطفي
                               ...                        
52598             wa7dek ya m3alem ena n7eb 8neyetek elkol
52599    bravo enchalah lbe9in yamlo kifo 5ater tonis t...
52600    yasir ta3jebni nhebha bercha fanana wa3liha le...
52601                yser mjodra w moch 5arej 3liha idawer
52602                                          wahdek olfa
Name: text, Length: 52603, dtype: object

In [197]:
####################################################################

In [204]:
emb_dim = 100

In [205]:
settings = {
 "size":emb_dim, 
  "alpha":0.05, 
    "window":5, 
    "min_count":4,
    "workers":8, 
    "min_alpha":0.0001, 
    "sg":1, 
    "hs":1, 
    "negative":0,
    "cbow_mean":0,
    "iter":15
}

In [206]:
#w2v_model = Word2Vec(corpus,min_count=5 ,size=emb_dim ,window=5, alpha=0.01)
w2v_model = Word2Vec(corpus,**settings)

In [207]:
from time import time
t = time()

w2v_model.train(corpus, total_examples=w2v_model.corpus_count, epochs=50, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

Time to train the model: 0.56 mins


In [208]:
w2v_model.wv.vectors.shape

(11426, 100)

In [209]:
w2v_model.wv.save_word2vec_format("Word2Vec_"+str(emb_dim))


In [37]:
####################################################################

In [210]:
w_matrix = w2v_model.wv.vectors
w_matrix.shape

(11426, 100)

In [211]:
#type(w2v_model.wv.index2entity)
meta_data = w2v_model.wv.index2entity

#w2v_model.wv.index2entity[0]
type(w2v_model.wv.index2word)
#w2v_model.wv.word_vec

#w2v_model.wv[w2v_model.wv.index2entity[0]]

#w_matrix[1000] == w2v_model.wv[w2v_model.wv.index2entity[1000]]

w2v_model.wv.most_similar(positive=["tofla"])

#w2v_model["rajel"] - w2v_model["mra"] - w2v_model["rjel"] + w2v_model["tofla"]

#w2v_model["khayeb"]

[('sotha', 0.5025343894958496),
 ('ma7leha', 0.49881526827812195),
 ('w', 0.49469253420829773),
 ('ama', 0.48090028762817383),
 ('tmathel', 0.4762348234653473),
 ('zeda', 0.46412956714630127),
 ('yeser', 0.4574049711227417),
 ('lawa7', 0.4555981457233429),
 ('barcha', 0.4549383521080017),
 ('ta7fona', 0.4530925750732422)]

In [218]:
from gensim.models import KeyedVectors
from gensim.test.utils import datapath
wv_from_text = KeyedVectors.load_word2vec_format(datapath('/home/mahmoud/Desktop/Tunisian Dialect Language Model TDLM/Sentiment analysis/Word2Vec Embeddings/Word2Vec_100'), binary=False)

In [219]:
wv_from_text.most_similar(positive=["امراه","راجل"],negative=["ولد"])

[('كافون', 0.4329281151294708),
 ('محترمه', 0.430868923664093),
 ('مرا', 0.39773425459861755),
 ('يوفقها', 0.3764894902706146),
 ('لثقافه', 0.3656095862388611),
 ('الماضيه', 0.35368573665618896),
 ('انتي', 0.34869128465652466),
 ('سياسيه', 0.34168222546577454),
 ('النواب', 0.3416549861431122),
 ('يازعيم', 0.3369888663291931)]

In [220]:
w2v_model.wv.most_similar(positive=["امراه","راجل"],negative=["ولد"])

[('كافون', 0.4329281151294708),
 ('محترمه', 0.430868923664093),
 ('مرا', 0.39773425459861755),
 ('يوفقها', 0.3764894902706146),
 ('لثقافه', 0.3656095862388611),
 ('الماضيه', 0.35368573665618896),
 ('انتي', 0.34869128465652466),
 ('سياسيه', 0.34168222546577454),
 ('النواب', 0.3416549861431122),
 ('يازعيم', 0.3369888663291931)]

In [221]:
wv_from_text.similarity("معلم","تحفون")

0.38524628

In [222]:
wv_from_text.similarity("معلم","خايب")

0.10449227

In [223]:
wv_from_text.similarity("كره","ممثل")

-0.061423913

data = pd.read_csv("/home/mahmoud/Desktop/PFE_Tunisian_Dialect/data/test_labeled_processed.csv")
data.drop(axis=1,columns=["Unnamed: 0","words"],inplace=True)
data["text"] = data["text"].apply(str)


data["text"] = data["text"].apply(remove_emojis)
data["text"] = data["text"].apply(clean_text)
data["text"] = data["text"].apply(remove_repeating_char)
data["text"] = data["text"].apply(normalize_latin)
data["text"] = data["text"].apply(normalize_arabic)
data["text"] = data["text"].apply(keep_letters_only)
data.head()

data.to_csv("/home/mahmoud/Desktop/PFE_Tunisian_Dialect/data/test.csv",index=False)

In [228]:
'/home/mahmoud/Desktop/Tunisian Dialect Language Model TDLM/Sentiment analysis/Word2Vec Embeddings/Word2Vec_100'
w2v_path = '/home/mahmoud/Desktop/Tunisian Dialect Language Model TDLM/Sentiment analysis/Word2Vec Embeddings/Word2Vec_150'

In [229]:
from gensim.models import KeyedVectors

# Load gensim word2vec
#w2v_path = '/home/mahmoud/Desktop/Tunisian Dialect Language Model TDLM/Sentiment analysis/Word2Vec Embeddings/Word2Vec_100'
w2v = KeyedVectors.load_word2vec_format(w2v_path)

import io

# Vector file, `\t` seperated the vectors and `\n` seperate the words
"""
0.1\t0.2\t0.5\t0.9
0.2\t0.1\t5.0\t0.2
0.4\t0.1\t7.0\t0.8
"""
out_v = io.open('vecs_SG150.tsv', 'w', encoding='utf-8')

# Meta data file, `\n` seperated word
"""
token1
token2
token3
"""
out_m = io.open('meta_SG150.tsv', 'w', encoding='utf-8')

# Write meta file and vector file
for index in range(len(w2v.index2word)):
    word = w2v.index2word[index]
    vec = w2v.vectors[index]
    out_m.write(word + "\n")
    out_v.write('\t'.join([str(x) for x in vec]) + "\n")
out_v.close()
out_m.close()